In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *



In [2]:
spark = SparkSession.builder \
                    .master('local[4]')\
                    .appName('join')\
                    .getOrCreate()

24/04/11 22:43:18 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.25 instead (on interface wlp0s20f3)
24/04/11 22:43:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/11 22:43:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/11 22:43:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
employee_df = spark.read.format('csv')\
                        .option('inferschema',True)\
                        .option('mode','permissive')\
                        .option('header',True)\
                        .option("escape", '\"')\
                        .load('employee.csv')
employee_df.show()

+-----------+--------------+-----------------+-------------+
|employee_id|          name|        job_title|department_id|
+-----------+--------------+-----------------+-------------+
|          1|      John Doe|Software Engineer|          101|
|          2|    Jane Smith|   Data Scientist|          102|
|          3|   Bob Johnson|  Product Manager|          101|
|          4|Alice Williams|Software Engineer|          103|
|          5|   David Brown|     Data Analyst|          102|
|          6|   Emily Davis|Software Engineer|          101|
|          7|  Chris Miller|  Product Manager|          103|
|          8|  Susan Wilson|   Data Scientist|          102|
|          9|    Mike Jones|Software Engineer|          101|
|         10|   Laura White|     Data Analyst|          102|
+-----------+--------------+-----------------+-------------+



In [4]:
sc = spark.sparkContext
rdd = sc

In [5]:
dept_df = spark.read.format('csv')\
                        .option('inferschema',True)\
                        .option('mode','permissive')\
                        .option('header',True)\
                        .option("escape", '\"')\
                        .load('department.csv')
dept_df.show()
                        

+-------------+------------------+
|department_id|   department_name|
+-------------+------------------+
|          101|       Engineering|
|          102|      Data Science|
|          103|Product Management|
|          104| Quality Assurance|
|          105|         Marketing|
+-------------+------------------+



In [6]:
join_df = employee_df.alias("e").join(dept_df.alias("d"), on =col("e.department_id") == col('d.department_id'), how = 'inner').select("e.employee_id", "e.name", "d.department_name")
join_df.show()

+-----------+--------------+------------------+
|employee_id|          name|   department_name|
+-----------+--------------+------------------+
|          1|      John Doe|       Engineering|
|          2|    Jane Smith|      Data Science|
|          3|   Bob Johnson|       Engineering|
|          4|Alice Williams|Product Management|
|          5|   David Brown|      Data Science|
|          6|   Emily Davis|       Engineering|
|          7|  Chris Miller|Product Management|
|          8|  Susan Wilson|      Data Science|
|          9|    Mike Jones|       Engineering|
|         10|   Laura White|      Data Science|
+-----------+--------------+------------------+

